# Прогноз дохода клиентов

 - Задача прогнозирования дохода имеет особенное значение в Банке.
Эта информация помогает точнее и более релевантно подбирать продукты и условия их приобретения, что в свою очередь вносит существенный вклад в прибыль Банка.
 - Помимо ценности для Банка, оценка дохода является регуляторным требованием ЦБ в части расчета предельно допустимой кредитной нагрузки для клиента (далее ПДН)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# здесь меняю рабочую папку, чтобы путь не прописывать к ней все время
import os
for dirpath, dirnames, _ in os.walk('/content/drive/MyDrive'):
    if 'hackathon' in dirnames:
        os.chdir(os.path.join(dirpath, 'hackathon'))
        break

# Импорт библиотек

In [11]:
import pandas as pd
import numpy as np

In [12]:
def weighted_mean_absolute_error(y_true, y_pred, weights):
    return (weights * np.abs(y_true - y_pred)).mean()

# Пример расчета метрики качества

In [31]:
train_df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';')
test_df = pd.read_csv('hackathon_income_test.csv', decimal=',', sep=';')
train_df.shape, test_df.shape

/tmp/ipython-input-3956870551.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';')
/tmp/ipython-input-3956870551.py:2: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('hackathon_income_test.csv', decimal=',', sep=';')


((76786, 224), (73214, 222))

In [32]:
train_df.head(5)

,id,dt,target,turn_cur_cr_avg_act_v2,salary_6to12m_avg,hdb_bki_total_max_limit,dp_ils_paymentssum_avg_12m,hdb_bki_total_cc_max_limit,incomeValue,gender,...,dp_ils_uniq_companies_1y,avg_6m_travel,avg_6m_government_services,hdb_bki_active_cc_max_overdue,total_rur_amt_cm_avg_period_days_ago_v2,label_Above_1M_share_r1,transaction_category_supermarket_sum_cnt_d15,max_balance_rur_amt_1m_af,w,first_salary_income
0,2,2024-04-30,109324.476325,1465144.960,NaN,52800.0,365346.244634,23213.0,97366.0,Женский,...,1.0,0.0,57.0,0.0,297.0,0.027027,9.0,NaN,0.301217,NaN
1,4,2024-02-29,25558.028662,303593.660,NaN,260200.0,NaN,10000.0,32580.0,Женский,...,NaN,0.0,707.0,67.0,30245.0,NaN,2.0,NaN,0.695800,NaN
2,5,2024-02-29,40666.753098,490754.010,NaN,2000000.0,NaN,90000.0,96866.0,Женский,...,NaN,422.0,0.0,0.0,210322.0,0.000000,20.0,NaN,0.515970,NaN
3,6,2024-04-30,43856.672058,219875.120,NaN,75000.0,NaN,75000.0,43860.0,Мужской,...,NaN,0.0,0.0,0.0,7187.0,0.000000,7.0,NaN,0.478003,NaN
4,7,2024-04-30,130420.851992,1750241.845,NaN,1000000.0,NaN,240000.0,83815.0,Женский,...,NaN,0.0,84.0,0.0,690038.0,0.000000,9.0,NaN,0.552314,NaN


In [33]:
X = train_df.drop(columns=['target'])
y = train_df['target']

In [35]:
print(f"Кол-во столбцов {X.shape[1]}")
object_cols = X.columns[X.dtypes == 'object']
print(f"Кол-во категориальных столбцов {len(object_cols)}")
str_cols = X.columns[X.dtypes == 'string']
print(f"Кол-во строковых столбцов {len(str_cols)}")
int_rows = X.columns[X.dtypes == 'int64']
print(f"Кол-во целочисленных столбцов {len(int_rows)}")
float_rows = X.columns[X.dtypes == 'float64']
print(f"Кол-во вещественных столбцов {len(float_rows)}")

Кол-во столбцов 223
Кол-во категориальных столбцов 41
Кол-во строковых столбцов 0
Кол-во целочисленных столбцов 1
Кол-во вещественных столбцов 181


In [36]:
X.drop(columns=['id'], inplace=True) # этот признак нам не нужен

In [77]:
X = X.loc[:, X.isnull().mean() < 0.5] # оставим только столбцы, где пропусков меньше 50%

# Submission

In [16]:
test_df['target'] = np.random.normal(mean, std, test_df.shape[0])
test_df[['id','target']].set_index('id').to_csv('submission.csv', decimal='.', sep=',')